In [1]:
import pandas as pd
import numpy as np
import librosa
import matplotlib.pyplot as plt
import os
from IPython.display import clear_output
from IPython.display import Audio

taxonomy = pd.read_csv('BirdCLEF/eBird_Taxonomy_v2021.csv')
metadata = pd.read_csv('BirdCLEF/train_metadata.csv')

import keras
model_path = 'BirdNET-Analyzer/checkpoints/V2.3/BirdNET_GLOBAL_3K_V2.3_Model'
model = keras.models.load_model(model_path, compile=False).model

2023-05-28 18:36:51.649189: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.1 when it was built against 1.14.0, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [4]:
f = metadata[metadata.primary_label == 'broman1'].filename.values[0]
sig, _ = librosa.load('BirdCLEF/train_audio/%s' % f, sr = 48000, res_type = 'kaiser_fast')
spec = librosa.feature.melspectrogram(y = sig, sr = 48000, fmin = 150, fmax = 15000, n_fft = 1024, hop_length = 282, n_mels = 96, norm = 100.0, power = 0.6)

In [7]:
chunk_len = 3*48000
subdiv = 8
samples_per_subunit = chunk_len // subdiv
num_subunits = sig.shape[0] // samples_per_subunit
tmp = sig[0:num_subunits*samples_per_subunit].reshape(num_subunits, samples_per_subunit)
chunks = np.array([tmp[i:num_subunits-subdiv+1+i] for i in range(subdiv)]).transpose(1,0,2).reshape(-1,chunk_len)
chunks.shape

(164, 144000)

In [8]:
pred = model.predict(chunks)

6/6 [==============================] - 8s 1s/step


In [12]:
intermediate_model = keras.Model(inputs = model.input, outputs = model.get_layer('ACT_POST').output)
intermediate = intermediate_model.predict(chunks)

6/6 [==============================] - 7s 1s/step


In [16]:
intermediate.shape

(164, 3, 8, 192)

In [27]:
np.max(np.abs(intermediate[1,:,7,:] - intermediate[7,:,1,:]))

4.3154616